In [1]:
import numpy as np
import pandas as pd

from scipy.stats import skew, kurtosis
from scipy.optimize import minimize

In [2]:
# Load the CSV file back into a dataframe
# returns_df = pd.read_csv('./data/generated_asset_returns.csv', index_col=0)
returns_df = pd.read_csv('./data/portfolio_returns.csv', index_col=0)
display(returns_df.head())

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,-0.020257,0.004057,0.004035,0.019693,0.000179,0.009305,0.003678,0.005784,0.009483,-0.011953,...,0.015881,0.000212,0.028237,0.009758,0.006987,0.017539,-0.001730,0.002409,0.013735,-0.010857
2016-01-06,-0.114863,-0.015879,0.002412,-0.017556,-0.007727,-0.012473,-0.001736,-0.011239,-0.035867,-0.009551,...,0.005547,0.000212,0.001592,-0.015647,0.003108,-0.010155,-0.007653,-0.030048,-0.009035,-0.029145
2016-01-07,-0.051389,-0.041922,-0.016573,-0.027699,-0.011047,-0.019769,-0.012206,-0.008856,-0.046058,-0.025394,...,-0.022066,-0.030310,-0.010411,-0.031557,-0.016148,-0.002700,-0.022845,-0.020570,-0.005492,-0.030019
2016-01-08,0.002736,-0.022705,-0.016036,-0.025425,0.001098,-0.002241,0.005707,-0.016403,-0.017642,-0.001649,...,-0.001539,-0.011366,-0.007308,-0.001448,0.000896,-0.033839,-0.001117,-0.011386,-0.009719,-0.011254
2016-01-11,-0.043383,0.001693,-0.016851,-0.010215,0.000915,-0.011791,0.005674,0.005287,0.006616,0.000330,...,0.016435,0.000000,0.009869,-0.001450,0.012224,0.014570,0.005367,-0.004607,0.005800,-0.019919


In [3]:
returns = returns_df.to_numpy()
print(returns)

[[-0.02025699  0.00405728  0.0040354  ...  0.00240942  0.01373474
  -0.01085703]
 [-0.11486289 -0.01587861  0.0024119  ... -0.03004779 -0.00903456
  -0.02914472]
 [-0.05138894 -0.04192186 -0.0165734  ... -0.02057019 -0.00549238
  -0.03001945]
 ...
 [-0.00143013  0.01456911  0.00173238 ...  0.00259785  0.00075938
   0.00158707]
 [-0.00071628 -0.01198273  0.00507206 ...  0.00388618 -0.00682872
   0.00538746]
 [-0.03045487 -0.00466462 -0.00848739 ... -0.00412956 -0.0021007
  -0.00504343]]


In [4]:
# Calculate moments
num_assets = returns.shape[1]
mean = np.mean(returns, axis=0)
cov = np.cov(returns.T)
centered_returns = returns - mean

In [5]:
co_skewness = np.zeros((num_assets, num_assets, num_assets))
for i in range(num_assets):
    for j in range(num_assets):
        for k in range(num_assets):
            co_skewness[i, j, k] = np.mean(centered_returns[:, i] * centered_returns[:, j] * centered_returns[:, k])        

In [6]:
co_kurtosis = np.zeros((num_assets, num_assets, num_assets, num_assets))
for i in range(num_assets):
    for j in range(num_assets):
        for k in range(num_assets):
            for l in range(num_assets):
                co_kurtosis[i, j, k, l] = np.mean(centered_returns[:, i] * centered_returns[:, j] * centered_returns[:, k] * centered_returns[:, l])

In [7]:
print("Number of assets:", returns.shape[1])
print("Mean Vector:\n", mean)
print("\nCovariance Matrix:\n", cov)

# Note: Printing the full co-skewness and co-kurtosis matrices would be very large,
# so we'll just show their shape to demonstrate they've been created.
print("\nCo-skewness matrix shape:", co_skewness.shape)
print("Co-kurtosis matrix shape:", co_kurtosis.shape)

Number of assets: 25
Mean Vector:
 [0.00065105 0.00070095 0.00071345 0.00019615 0.00059776 0.00060434
 0.00013207 0.00125817 0.00114446 0.00091131 0.00087903 0.00030145
 0.00092836 0.00021806 0.00141724 0.00047391 0.00068585 0.00052017
 0.00054732 0.00019592 0.00104882 0.00117368 0.00068171 0.00032944
 0.0009084 ]

Covariance Matrix:
 [[ 1.68485760e-03  5.36275897e-04  1.30587850e-04  1.18659808e-04
   1.93395090e-04  2.62594119e-04 -3.23435390e-06  3.14726159e-04
   3.70492069e-04  2.64363304e-04  3.82389465e-04  3.32979837e-04
   1.80292540e-04  1.47360274e-04  1.91333111e-04  1.07772923e-04
   2.65107429e-04  4.70493817e-05  2.50832907e-04  1.84251934e-04
   1.17453290e-04  1.24351787e-04  4.48692933e-04  7.31374012e-05
   4.47282758e-04]
 [ 5.36275897e-04  7.71157910e-04  1.13256467e-04  1.19459851e-04
   1.79124983e-04  2.58217453e-04  1.37364285e-05  2.94261552e-04
   2.78073606e-04  2.21697488e-04  3.10437606e-04  3.76049783e-04
   1.64151873e-04  1.53280386e-04  1.88260456e-04 

In [8]:
def objective_function(weights, mean, cov, co_skewness, co_kurtosis, A, B, C):
    """
    Calculates a portfolio objective function with higher order moments.
    """
    mean_term = weights.T @ mean
    # variance_term = weights.T @ cov @ weights
    variance_term = np.dot(weights.flatten(), cov @ weights.flatten())
    # skewness_term = weights.T @ co_skewness @ (weights * weights)
    skewness_term = np.einsum('i,j,k,ijk->', weights.flatten(), weights.flatten(), weights.flatten(), co_skewness)
    # kurtosis_term = weights.T @ co_kurtosis @ (weights * weights * weights)
    kurtosis_term = np.einsum('i,j,k,l,ijkl->', weights.flatten(), weights.flatten(), weights.flatten(), weights.flatten(), co_kurtosis)


    return (-mean_term + A / 2 * variance_term - B / 6 * skewness_term + C / 24 * kurtosis_term)

In [9]:
initial_weights = np.ones(num_assets) / num_assets
print(initial_weights)

[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]


In [10]:
A = 2
B = -1
C = -1

In [11]:
# Call the objective function
result = objective_function(initial_weights, mean, cov, co_skewness, co_kurtosis, A, B, C)

print(f"The calculated objective function value is: {result:.6f}")

The calculated objective function value is: -0.000552


In [12]:
objective_args = (mean, cov, co_skewness, co_kurtosis, A, B, C)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
bounds = tuple((0, 1) for _ in range(num_assets))

In [13]:
result = minimize(
    objective_function,
    initial_weights,
    args = objective_args,
    method = 'SLSQP',
    bounds = bounds,
    constraints = constraints    
)

In [14]:
# Print the results
if result.success:
    optimal_weights = result.x.reshape(-1, 1)
    optimal_value = result.fun
    print(f"Optimization successful!")
    print("\nOptimal Weights:")
    for i in range(num_assets):
        print(f"Asset {i}: {optimal_weights[i][0]:.4f}")
    print(f"\nMinimum objective function value: {optimal_value:.6f}")
else:
    print("Optimization failed.")
    print(result.message)

Optimization successful!

Optimal Weights:
Asset 0: 0.0000
Asset 1: 0.0000
Asset 2: 0.0000
Asset 3: 0.0000
Asset 4: 0.0000
Asset 5: 0.0000
Asset 6: 0.0000
Asset 7: 0.1788
Asset 8: 0.1114
Asset 9: 0.0354
Asset 10: 0.0000
Asset 11: 0.0000
Asset 12: 0.0711
Asset 13: 0.0000
Asset 14: 0.2814
Asset 15: 0.0000
Asset 16: 0.0000
Asset 17: 0.0000
Asset 18: 0.0000
Asset 19: 0.0000
Asset 20: 0.1308
Asset 21: 0.1912
Asset 22: 0.0000
Asset 23: 0.0000
Asset 24: 0.0000

Minimum objective function value: -0.001050


In [15]:
weights = optimal_weights

print(f"Mean term: {(weights.T @ mean).item():.5f}")
print(f"Variance term: {(A / 2 * weights.T @ cov @ weights).item():.5f}")
print(f"Skewness term: {B / 6 * np.einsum('i,j,k,ijk->', weights.flatten(), weights.flatten(), weights.flatten(), co_skewness):.5f}")
print(f"Kurtosis term: {C / 24 * np.einsum('i,j,k,l,ijkl->', weights.flatten(), weights.flatten(), weights.flatten(), weights.flatten(), co_kurtosis):.5f}")

Mean term: 0.00121
Variance term: 0.00016
Skewness term: 0.00000
Kurtosis term: -0.00000


In [ ]:
def generate_skewed_fat_tailed_returns(num_assets, num_periods, mean_vector, cov_matrix, degrees_of_freedom=4, skewness_factor=0.3):
    """
    Generates a returns array with skewed and fat-tailed characteristics.

    Args:
        num_assets (int): The number of assets.
        num_periods (int): The number of time periods (e.g., trading days).
        mean_vector (np.ndarray): The mean returns vector (1 x num_assets).
        cov_matrix (np.ndarray): The covariance matrix (num_assets x num_assets).
        degrees_of_freedom (int): Degrees of freedom for the t-distribution.
                                  Lower values produce fatter tails.
        skewness_factor (float): Factor to introduce negative skewness.
                                 Higher values mean more skewness.

    Returns:
        np.ndarray: A num_periods x num_assets array of synthetic returns.
    """
    # 1. Generate data with fat tails using a multivariate t-distribution approach.
    # We simulate from a standard normal distribution...
    standard_normal_data = np.random.multivariate_normal(
        np.zeros(num_assets),
        cov_matrix,
        size=num_periods
    )

    # ...and then scale it by a random variable from a chi-squared distribution.
    # This process, when the mean is added, creates a multivariate t-distribution,
    # which has fatter tails than a normal distribution.
    chi_squared_rv = np.random.chisquare(degrees_of_freedom, size=num_periods)
    fat_tailed_data = standard_normal_data * np.sqrt(degrees_of_freedom / chi_squared_rv)[:, np.newaxis]

    # 2. Add skewness by introducing a "bad day" scenario.
    # A simple way to do this is to add a large negative shock with a small probability.
    is_bad_day = np.random.rand(num_periods, 1) < skewness_factor
    skew_shock = np.random.uniform(low=-0.05, high=0, size=(num_periods, num_assets)) * is_bad_day
    
    # 3. Add the mean and combine the terms.
    # The final returns are the fat-tailed data plus the mean and the skewness shock.
    returns = fat_tailed_data + mean_vector + skew_shock

    return returns

In [ ]:
# Define the number of assets and periods
num_assets = 10
num_periods = 252

# Create a sample mean vector and covariance matrix
# Mean returns for each of the 10 assets (annualized).
sample_mean = np.array([0.08, 0.12, 0.05, 0.10, 0.07, 0.09, 0.11, 0.06, 0.15, 0.08])
# The covariance matrix can be more complex, but we'll create a simple one here.
# A simple way is to start with a correlation matrix and then scale by standard deviations.
# Let's create a diagonal matrix for simplicity (assuming zero correlation).
sample_cov = np.diag(np.ones(num_assets) * 0.02**2) # assuming 20% annualized volatility

# Let's set a slightly more realistic covariance matrix with some correlation
np.random.seed(42)
asset_vols = np.random.uniform(0.15, 0.35, num_assets)
correlation_matrix = np.eye(num_assets) * 0.5 + 0.5 # A simple way to create a correlation matrix.
correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2
np.fill_diagonal(correlation_matrix, 1)
sample_cov = np.outer(asset_vols, asset_vols) * correlation_matrix

# Generate the returns
generated_returns = generate_skewed_fat_tailed_returns(
    num_assets=num_assets,
    num_periods=num_periods,
    mean_vector=sample_mean,
    cov_matrix=sample_cov
)

# Print the shape and the first 5 rows of the generated returns
print(f"Generated returns array shape: {generated_returns.shape}\n")
print("First 5 rows of the generated returns:")
print(generated_returns[:5, :])

# Save the generated returns to a CSV file with a date index
output_filename = 'generated_asset_returns.csv'

# Create a pandas DataFrame from the NumPy array
column_names = [f'Asset_{i+1}' for i in range(num_assets)]
dates = pd.date_range(start='2024-01-01', periods=num_periods, freq='B') # 'B' for business days
df = pd.DataFrame(generated_returns, index=dates, columns=column_names)

# Save the DataFrame to a CSV file
df.to_csv(output_filename, index=True)
print(f"\nSuccessfully saved the data to '{output_filename}' with a date index.")